In [25]:
"""
RAG的基本流程和方法
1、使用langchain的document_load来加载文本
2、使用text_split来分割文本
3、加载为document类型数据
4、embedding成向量
5、使用chorma和fassio来存储向量（非持久性，存在内存中）
6、可以将向量存在splite中做永久性储存
"""

from langchain_community.llms.ollama import Ollama
from langchain_community.chat_models.ollama import ChatOllama

llm = Ollama(model = "glm4")

In [1]:
from langchain_community.chat_models import ChatZhipuAI
import os
os.environ["ZHIPUAI_API_KEY"] = ""
model = ChatZhipuAI(model="glm-4",streamer = True)

In [1]:
from langchain.llms.base import LLM
from typing import Optional, List, Mapping, Any
import requests

class local_model(LLM):
    history = []
    def __init__(self):
        super().__init__()
        
    @property
    def _llm_type(self) -> str:
        return "local_model"

    def _call(self, prompt: str, stop: Optional[List[str]] = None) -> str:
        data={'text':prompt}
        url = "http://0.0.0.0:6667/chat/"
        response = requests.post(url, json=data)
        if response.status_code!=200:
            return "error"
        resp = response.json()
        if stop is not None:
            response = enforce_stop_tokens(response, stop)
        self.history = self.history+[[None, resp['result']]]
        return resp['result']

In [4]:
'''RAG的utils'''
import jsonlines
from langchain.indexes import SQLRecordManager, index
from langchain.text_splitter import RecursiveJsonSplitter,CharacterTextSplitter
import sentence_transformers
from typing import List
from langchain_core.documents import Document
from langchain.vectorstores.chroma import Chroma
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PDFMinerLoader
import os

def acq_filenames(folder_path):
    filename_list = []
    for filename in os.listdir(folder_path):
        filename_list.append(folder_path + "/" + filename)
    return filename_list

def read_jsonl(jsonl_file):
    new_dict = {}
    with open(jsonl_file,"r",encoding="utf-8") as jsonfile:
        for item in jsonlines.Reader(jsonfile):
            for key, value in item.items():
                new_dict[key] = value
    return new_dict

def json2doc(file_path):
    json_data = read_jsonl(text_path)

    splitter = RecursiveJsonSplitter(max_chunk_size=500)
    # 递归拆分json数据 - 如果需要访问/操作较小的json块
    # json_chunks = splitter.split_json(json_data=json_data)
    # 拆分器还可以输出文档
    docs = splitter.create_documents(texts=[json_data])
    return docs

def pdf2doc(file_path):

    docs = PDFMinerLoader(file_path).load()
    
    return docs
    
    

def text2embedding(docs):
    # 把文档进行分块
    chunks = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=10, add_start_index=True).split_documents(documents=docs)
    
    # 向量化
    embedding_model = ""
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model, model_kwargs={'device': "cuda:1"})
    embeddings.client = sentence_transformers.SentenceTransformer(embeddings.model_name, device="cuda:1")
    
    vector_store = Chroma.from_documents(documents=chunks, embedding=embeddings)
    
    return vector_store
    
def permanent_stored(chunks):
    chunks = CharacterTextSplitter(separator="\n", chunk_size=1000, chunk_overlap=10, add_start_index=True).split_documents(documents=docs)
    # 向量化
    embedding_model = ""
    embeddings = HuggingFaceEmbeddings(model_name=embedding_model, model_kwargs={'device': "cuda:1"})
    embeddings.client = sentence_transformers.SentenceTransformer(embeddings.model_name, device="cuda:1")

    vectorstore = Chroma.from_documents(
        documents=chunks, embedding=embeddings,
        persist_directory='chroma_db_demo'  # 存储db地址
                )

    return vectorstore



In [3]:
'''stored embedding'''
from langchain.document_loaders.pdf import PDFMinerLoader
from langchain_community.document_loaders import JSONLoader
import time
from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from transformers import AutoTokenizer, AutoModelForCausalLM
import os
import torch
from tqdm.notebook import tqdm
from time import sleep


# 使用json
# text_path = "./text_file/output.jsonl"
file_list = acq_filenames("poem")
docs = []

for file in tqdm(file_list):

    if file.endswith(".jsonl"):
        doc = json2doc(file)
        for item in doc:
            docs.append(item)
    elif file.endswith(".pdf"):
        doc = pdf2doc(file)
        for item in doc:
            docs.append(item) 





  0%|          | 0/7 [00:00<?, ?it/s]

In [5]:
vector_store = permanent_stored(docs)

/home/weibs/miniconda3/envs/Pytorch/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(


In [6]:
'''查询'''
# %run model.ipynb
s = time.time()
# 获取Retriever

rag_retriver = vector_store.as_retriever(search_type="mmr", search_kwargs={'k': 3})

prompt_template = PromptTemplate.from_template("""
        你是一名对答如流的专家，请根据如下的内容回答问题
        {context}
        请如实地回答用户所提出的问题:{question}，如果遇到你不知道的（包含在上述内容中也没有提到的），请回复“我不知道”。
        """)


# 使用Expression Language语法组合成Chain
rag_chain = {"context": rag_retriver, "question": RunnablePassthrough()} | prompt_template | local_model()


# async for chunk in rag_chain.astream("诗礼乐教化有什么价值和困难"):
#     # print('-')  # noqa: T201
#     print(chunk.content, end='', flush=True)  # noqa: T201
# 提问&回答
response = rag_chain.invoke("诗礼乐教化有什么价值和困难")
 
print(response)
e = time.time()
print(e-s)


诗礼乐教化的价值主要体现在以下几个方面：

1. **情感浸润**：诗教通过诗歌表达情感，能够培养人的情感世界，使人们更加细腻地感受生活，增强同情心和审美能力。

2. **礼仪规范**：礼教强调行为规范和道德准则，有助于维护社会秩序，培养人的社会责任感和集体主义精神。

3. **人格塑造**：乐教通过音乐教育，塑造人的品格，培养人的道德情操，提升个人的精神境界。

4. **道德教化**：诗礼乐教化是道德教化的重要手段，有助于传承和弘扬中华民族优秀传统文化。

5. **社会和谐**：通过诗礼乐教化，可以促进社会成员之间的和谐相处，构建和谐社会。

然而，诗礼乐教化也面临一些困难和挑战：

1. **时代变迁**：随着社会的发展和变迁，传统的诗礼乐教化方式可能难以适应现代社会的需求。

2. **文化冲击**：外来文化的影响可能对传统的诗礼乐教化产生冲击，导致传统文化传承的困境。

3. **教育方式**：传统的诗礼乐教化方式可能过于注重形式，缺乏创新，难以激发学生的学习兴趣。

4. **教育资源**：诗礼乐教化需要丰富的教育资源，而在实际操作中，可能存在教育资源不足的问题。

至于上述内容中没有提到的困难，例如教育体制、社会环境等方面的因素，我无法给出具体回答。因此，对于这些未知的内容，我的回复是“我不知道”。
22.341047763824463
